In [39]:
import pandas as pd
import requests, json
from glob import glob

In [40]:
raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=vaccination_data')
loadedJson = raw.json()['vaccination_data']
vaccinationData = pd.DataFrame(loadedJson)

with open(f'./json/cdc_vaccine_data_{loadedJson[0]["Date"]}.json', 'w') as outfile:
    json.dump(loadedJson, outfile)

In [133]:
dateDf = pd.read_csv('../../docs/csv/covid_confirmed_1p3a_state.csv')
datesList = list(dateDf.columns[2:])
datesList.sort()
datesList = ['GEOID', 'NAME'] + datesList
placeholderDf = pd.DataFrame(datesList).set_index(0).T

for idx, file in enumerate(vaccinationDataList):
    with open(file) as f:
        data = json.load(f)
        if (type(data)==dict):
            data = data['vaccination_data']
    currDate = data[0]['Date']
    vaccinationDf = pd.DataFrame(data) \
        .merge(geoidTable, left_on="Location", right_on="STUSPS", how="inner")[['GEOID','NAME','Doses_Distributed','Doses_Administered']]
    
    if idx == 0:
        vaccineAdministered = vaccinationDf[['GEOID','NAME','Doses_Administered']]
        vaccineDistributed = vaccinationDf[['GEOID','NAME','Doses_Distributed']]
        vaccineAdministered.columns = ['GEOID','NAME',currDate]
        vaccineDistributed.columns = ['GEOID','NAME',currDate]
    else:
        dailyVaccineAdministered = vaccinationDf[['GEOID','NAME','Doses_Administered']]
        dailyVaccineDistributed = vaccinationDf[['GEOID','NAME','Doses_Distributed']]
        dailyVaccineAdministered.columns = ['GEOID','NAME',currDate]
        dailyVaccineDistributed.columns = ['GEOID','NAME',currDate]
        
        vaccineAdministered = vaccineAdministered.merge(dailyVaccineAdministered, on=["GEOID","NAME"])
        vaccineDistributed = vaccineDistributed.merge(dailyVaccineDistributed, on=["GEOID","NAME"])
        
merged = pd.concat([placeholderDf, vaccineAdministered])
cols = list(merged.columns)[-2:] + list(merged.columns)[:-2]
merged = merged[cols]
merged.to_csv()

merged.to_csv('./csv/vaccine_admin_cdc_1p3a_state.csv', index=False)
merged.to_csv('../../docs/csv/vaccine_admin_cdc_1p3a_state.csv', index=False)

merged = pd.concat([placeholderDf, vaccineDistributed])
cols = list(merged.columns)[-2:] + list(merged.columns)[:-2]
merged = merged[cols]
merged.to_csv()

merged.to_csv('./csv/vaccine_dist_cdc_1p3a_state.csv', index=False)
merged.to_csv('../../docs/csv/vaccine_dist_cdc_1p3a_state.csv', index=False)

<ipython-input-133-e8ab5655eada>:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merged = pd.concat([placeholderDf, vaccineAdministered])
<ipython-input-133-e8ab5655eada>:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merged = pd.concat([placeholderDf, vaccineDistributed])


In [134]:
dateDf = pd.read_csv('../../docs/csv/covid_confirmed_nyt_state.csv')
datesList = list(dateDf.columns[1:])
datesList.sort()
datesList = ['fips'] + datesList
placeholderDf = pd.DataFrame(datesList).set_index(0).T

for idx, file in enumerate(vaccinationDataList):
    with open(file) as f:
        data = json.load(f)
        if (type(data)==dict):
            data = data['vaccination_data']
    currDate = data[0]['Date']
    vaccinationDf = pd.DataFrame(data) \
        .merge(geoidTable, left_on="Location", right_on="STUSPS", how="inner")[['GEOID','NAME','Doses_Distributed','Doses_Administered']]
    
    if idx == 0:
        vaccineAdministered = vaccinationDf[['GEOID','Doses_Administered']]
        vaccineDistributed = vaccinationDf[['GEOID','Doses_Distributed']]
        vaccineAdministered.columns = ['fips',currDate]
        vaccineDistributed.columns = ['fips',currDate]
    else:
        dailyVaccineAdministered = vaccinationDf[['GEOID','Doses_Administered']]
        dailyVaccineDistributed = vaccinationDf[['GEOID','Doses_Distributed']]
        dailyVaccineAdministered.columns = ['fips',currDate]
        dailyVaccineDistributed.columns = ['fips',currDate]
        
        vaccineAdministered = vaccineAdministered.merge(dailyVaccineAdministered, on=["fips"])
        vaccineDistributed = vaccineDistributed.merge(dailyVaccineDistributed, on=["fips"])
        
merged = pd.concat([placeholderDf, vaccineAdministered])
cols = list(merged.columns)[-1:] + list(merged.columns)[:-1]
merged = merged[cols]
merged.to_csv()

merged.to_csv('./csv/vaccine_admin_cdc_nyt_state.csv', index=False)
merged.to_csv('../../docs/csv/vaccine_admin_cdc_nyt_state.csv', index=False)

merged = pd.concat([placeholderDf, vaccineDistributed])
cols = list(merged.columns)[-1:] + list(merged.columns)[:-1]
merged = merged[cols]
merged.to_csv()

merged.to_csv('./csv/vaccine_dist_cdc_nyt_state.csv', index=False)
merged.to_csv('../../docs/csv/vaccine_dist_cdc_nyt_state.csv', index=False)

<ipython-input-134-6c298b75b39d>:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merged = pd.concat([placeholderDf, vaccineAdministered])
<ipython-input-134-6c298b75b39d>:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merged = pd.concat([placeholderDf, vaccineDistributed])


<hr>

In [4]:
raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_IL_external')
json = raw.json()['integrated_county_timeseries_external_data']
timeSeriesData = pd.DataFrame(json).sort_values('date')

In [31]:
filtered = timeSeriesData[['fips_code','date','report_date_window','report_date_window_start','percent_new_test_results_reported_positive_7_day_rolling_average','new_test_results_reported_7_day_rolling_average']]

In [32]:
filtered.head()

,fips_code,date,report_date_window,report_date_window_start,percent_new_test_results_reported_positive_7_day_rolling_average,new_test_results_reported_7_day_rolling_average
22224,17129,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
19448,17119,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
17723,17103,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
33705,17195,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
11512,17067,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0


In [36]:
filtered[filtered.fips_code == 17129][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
    .set_index('date').T

date,2020-01-22T00:00:00,2020-01-23T00:00:00,2020-01-24T00:00:00,2020-01-25T00:00:00,2020-01-26T00:00:00,2020-01-27T00:00:00,2020-01-28T00:00:00,2020-01-29T00:00:00,2020-01-30T00:00:00,2020-01-31T00:00:00,...,2020-12-23T00:00:00,2020-12-24T00:00:00,2020-12-25T00:00:00,2020-12-26T00:00:00,2020-12-27T00:00:00,2020-12-28T00:00:00,2020-12-29T00:00:00,2020-12-30T00:00:00,2020-12-31T00:00:00,2021-01-01T00:00:00
percent_new_test_results_reported_positive_7_day_rolling_average,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.42,5.99,6.34,6.78,7.35,8.37,8.3,8.22,10.08,11.24


In [1]:
import grequests

C:\Users\halpe\anaconda3\envs\trees\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
state2Digit = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [3]:
urls = [f"https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_{stateCode}_external" for stateCode in state2Digit]
breakpoint = 12
urlDict = []
for i in range(0,5):
    urlDict.append(urls[breakpoint*i:breakpoint*(i+1)])

In [4]:
responses = []
for urlList in urlDict:
    rs = (grequests.get(u,  timeout=120) for u in urlList)
    response = grequests.map(rs)
    responses.append(response)

In [8]:
parsed = ''

for responseSet in responses:
    for response in responseSet:
        if len(parsed)==0:
            parsed = pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])
        else:
            parsed = pd.concat([parsed, pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])])

parsed = parsed.sort_values('date')

In [9]:
uniqFips = list(parsed.fips_code.unique())

cleaned = ''

for fips in uniqFips:
    if len(cleaned) == 0:
        cleaned = parsed[parsed.fips_code == fips][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
            .set_index('date').T
        cleaned['fips'] = fips
    else:
        tempDf = parsed[parsed.fips_code == fips][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
            .set_index('date').T
        tempDf['fips'] = fips
        cleaned = pd.concat([cleaned, tempDf])
cleaned = cleaned[list(cleaned.columns)[-1:] + list(cleaned.columns)[:-1]]

In [10]:
columnNames = [col[:10] for col in cleaned.columns]

In [11]:
cleaned.columns = columnNames

In [12]:
cleaned.to_csv("./county_testing_data.csv",index=False)